<a href="https://colab.research.google.com/github/hhmida/ESB/blob/main/Exercice_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 53.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8e2b6f8efe7f22545b040e525b719941d1af2bd754e1df4b19f6b341c2ca808e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
!wget https://hhmida.github.io/tp-spark/assets/vente_export_der_ph_mois.csv
!wget https://hhmida.github.io/tp-spark/assets/prod_der_phosphate_mois.csv

--2022-12-12 13:33:02--  https://hhmida.github.io/tp-spark/assets/vente_export_der_ph_mois.csv
Resolving hhmida.github.io (hhmida.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to hhmida.github.io (hhmida.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6039 (5.9K) [text/csv]
Saving to: ‘vente_export_der_ph_mois.csv’

vente_export_der_ph 100%[===================>]   5.90K  --.-KB/s    in 0s      

2022-12-12 13:33:02 (71.2 MB/s) - ‘vente_export_der_ph_mois.csv’ saved [6039/6039]

--2022-12-12 13:33:03--  https://hhmida.github.io/tp-spark/assets/prod_der_phosphate_mois.csv
Resolving hhmida.github.io (hhmida.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to hhmida.github.io (hhmida.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7434 (7.3K) [text/csv]
Saving to: ‘prod_der_phosphate_mois.csv’

prod_der_phosphate_ 100%[==

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark =  SparkSession.builder.appName('Exercice Spark').getOrCreate()
sc = spark.sparkContext

In [4]:
from itertools import islice

In [5]:
rddP = sc.textFile('prod_der_phosphate_mois.csv').cache()
rddP = rddP.mapPartitionsWithIndex(lambda idx, iter: islice(iter,1,None) if (idx == 0) else iter)


In [6]:
rddV = sc.textFile('vente_export_der_ph_mois.csv').cache()
rddV = rddV.mapPartitionsWithIndex(lambda idx, iter: islice(iter,1,None) if (idx == 0) else iter)

# Q2.a

In [ ]:
def transformer(l):
  T=l.split(';')
  return int(T[0]),T[-1]

In [ ]:
rddP.map(transformer).sortByKey(False).collect()[0]

(8030, '31/08/2011')

## solution avec reduce

In [ ]:
def fmax(a,b):
  if a>b:
    return a
  return b

In [ ]:
rddP.map(transformer).reduce(max)

(8030, '31/08/2011')

#Q2.b

In [ ]:
def extract_year_prod(l):
  T=l.split(';')
  for i in range(len(T)-1):
    T[i]=int(T[i]) if T[i]!='' else 0
  return T[-1][-4:], T[:-1]

def somme_prod(a,b):
  resultat = []
  for i in range(len(a)):
    resultat.append(a[i]+b[i])
  return resultat

In [ ]:
q2b = rddP.map(extract_year_prod).reduceByKey(somme_prod).sortByKey()
q2b.collect()

[('2010',
  [52802, 23505, 143450, 0, 20799, 155150, 76430, 762722, 1276660, 1212633]),
 ('2011',
  [45270, 17151, 84660, 0, 8054, 79839, 54680, 408928, 428420, 518219]),
 ('2012',
  [31870, 14438, 116695, 0, 11022, 171737, 57480, 494311, 666025, 777012]),
 ('2013',
  [11260, 17620, 127449, 0, 9127, 131246, 46170, 518043, 823537, 747793]),
 ('2014',
  [21920, 7527, 124385, 0, 12366, 158112, 60470, 482314, 587528, 633027]),
 ('2015', [9670, 5836, 87060, 0, 5605, 149257, 67290, 261797, 369518, 475457]),
 ('2016',
  [12510, 11497, 67980, 0, 11582, 166339, 63160, 385043, 569800, 599085]),
 ('2017',
  [9383, 11614, 83730, 0, 12932, 170519, 29070, 299737, 564673, 597423]),
 ('2018',
  [20953, 1618, 57710, 0, 14454, 195247, 55750, 245436, 332831, 451945]),
 ('2019',
  [13085, 12427, 47660, 0, 12271, 160425, 35740, 276128, 444568, 435443]),
 ('2020',
  [7490, 1213, 46060, 39930, 4519, 61737, 28470, 52682, 334958, 310269])]

# Q2.C

In [ ]:
def extract_year_prod_total(l):
  T=l.split(';')
  for i in range(10):
    T[i]=int(T[i]) if T[i]!='' else 0
  return T[-1][-4:],sum(T[:10])

def somme_prod_total(a,b):
  return a+b

In [ ]:
q2c = rddP.map(extract_year_prod_total).reduceByKey(somme_prod_total).sortByKey()
q2c.collect()

[('2010', 3724151),
 ('2011', 1645221),
 ('2012', 2340590),
 ('2013', 2432245),
 ('2014', 2087649),
 ('2015', 1431490),
 ('2016', 1886996),
 ('2017', 1779081),
 ('2018', 1375944),
 ('2019', 1437747),
 ('2020', 887328)]

In [ ]:
q2cv2 = q2b.map(lambda a: (a[0], sum(a[1])))
q2cv2.collect()

[('2010', 3724151),
 ('2011', 1645221),
 ('2012', 2340590),
 ('2013', 2432245),
 ('2014', 2087649),
 ('2015', 1431490),
 ('2016', 1886996),
 ('2017', 1779081),
 ('2018', 1375944),
 ('2019', 1437747),
 ('2020', 887328)]

# Q2.d

Nous utilisons les mêmes fonctions créées dans Q2.b

In [ ]:
q2d = rddV.map(extract_year_prod).reduceByKey(somme_prod).sortByKey()
q2d.collect()

[('2010', [52000, 27322, 137202, 0, 54675, 712469, 1157379, 527250]),
 ('2011', [44253, 16954, 82675, 0, 28338, 361326, 354404, 249606]),
 ('2012', [33931, 14199, 108286, 3600, 34100, 446345, 558778, 360095]),
 ('2013', [7960, 10250, 126341, 6577, 20980, 484456, 634325, 284611]),
 ('2014', [21155, 7210, 121362, 1700, 42047, 524731, 503963, 247427]),
 ('2015', [11412, 7800, 81264, 0, 39500, 282673, 223786, 136077]),
 ('2016', [14242, 18915, 72627, 0, 36400, 385428, 560364, 297803]),
 ('2017', [9148, 8490, 75575, 0, 12000, 252480, 440065, 229448]),
 ('2018', [17075, 6450, 48911, 5000, 40356, 291571, 248846, 143407]),
 ('2019', [14300, 5753, 50573, 4000, 29563, 283223, 338336, 141739]),
 ('2020', [9150, 2420, 34843, 0, 15850, 53950, 320068, 126575])]

# Q2.e

In [ ]:
def difference_anne(l):
  annee = l[0]
  prod = l[1][0]
  ventes = l[1][1]
  resultat = []
  del prod[3], prod[4]
  for i in range(len(ventes)):
    resultat.append(prod[i]-ventes[i])
  return annee, resultat


In [ ]:
q2e = q2b.join(q2d).map(difference_anne)
q2e.collect()

[('2012', [-2061, 239, 8409, 7422, 23380, 47966, 107247, 416917]),
 ('2014', [765, 317, 3023, 10666, 18423, -42417, 83565, 385600]),
 ('2015', [-1742, -1964, 5796, 5605, 27790, -20876, 145732, 339380]),
 ('2017', [235, 3124, 8155, 12932, 17070, 47257, 124608, 367975]),
 ('2018', [3878, -4832, 8799, 9454, 15394, -46135, 83985, 308538]),
 ('2010', [802, -3817, 6248, 20799, 21755, 50253, 119281, 685383]),
 ('2011', [1017, 197, 1985, 8054, 26342, 47602, 74016, 268613]),
 ('2013', [3300, 7370, 1108, 2550, 25190, 33587, 189212, 463182]),
 ('2016', [-1732, -7418, -4647, 11582, 26760, -385, 9436, 301282]),
 ('2019', [-1215, 6674, -2913, 8271, 6177, -7095, 106232, 293704]),
 ('2020', [-1660, -1207, 11217, 4519, 12620, -1268, 14890, 183694])]

# Solutions avec DataFrame

In [7]:
schemaP = "NPK int,G27 int,STPP int,SSP int,NP int,NA int,DCP int, TSP int,DAP int, AP54 int,Year string"
schemaV = "NPK int,G27 int,STPP int,NA int,DCP int, TSP int,DAP int, AP54 int,Year string"

In [8]:
dfP = spark.read.csv('prod_der_phosphate_mois.csv',sep=';', header=True, schema=schemaP)
dfV = spark.read.csv('vente_export_der_ph_mois.csv', header=True, sep=';', schema=schemaV)

In [9]:
dfV.show()

+-----+----+-----+----+----+-----+------+-----+----------+
|  NPK| G27| STPP|  NA| DCP|  TSP|   DAP| AP54|      Year|
+-----+----+-----+----+----+-----+------+-----+----------+
| 2050|1607| 8186|null|2850|69490|134144|41277|31/01/2010|
| 4600|5150|11489|null|6985|24000| 46210|45322|28/02/2010|
| 6000|2300|14587|null|6540|68500| 86999|43274|31/03/2010|
| 1120|1905| 8974|null| 540|16500| 34051|52997|30/04/2010|
| 5600|6600|12861|null|4000|89984| 65808|43874|31/05/2010|
|    0|   0|12127|null|8024|68525|145938|11243|30/06/2010|
| 8780|1745|10436|null|5300|57900|127873|66579|31/07/2010|
| 6300|1640|12597|null|2701|78290|123132|54433|31/08/2010|
| 6000| 780|11168|null| 300|72340| 81026|38795|30/09/2010|
| 3400|5095|10820|null|6535|32930|114226|43246|31/10/2010|
| 4650| 500|11392|null|7500|61200|109884|31994|30/11/2010|
| 3500|   0|12565|null|3400|72810| 88088|54216|31/12/2010|
| 1800|   0|12240|null|   0|61878|108253|48821|31/01/2011|
| 2300|5850| 2570|null| 500| 4400| 13697|24518|28/02/201

In [10]:
dfV = dfV.fillna(0)
dfP = dfP.fillna(0)
dfV.createOrReplaceTempView('ventes')
dfP.createOrReplaceTempView('productions')

In [11]:
q3a = spark.sql('select year, npk from productions order by npk desc limit 1').show()

+----------+----+
|      year| npk|
+----------+----+
|31/08/2011|8030|
+----------+----+



In [12]:
q3a_v2=dfP.select('year','npk').orderBy('npk', ascending=False).limit(1)
q3a_v2.show()

+----------+----+
|      year| npk|
+----------+----+
|31/08/2011|8030|
+----------+----+



In [13]:
q3b = spark.sql('''select substr(year,7) as year, sum(NPK), sum(G27), sum(STPP), sum(SSP), sum(NP), 
                  sum(NA), sum(DCP), sum(TSP), sum(DAP), sum(AP54) from productions group by (substr(year,7)) order by 1''')
q3b.show()

+----+--------+--------+---------+--------+-------+-------+--------+--------+--------+---------+
|year|sum(NPK)|sum(G27)|sum(STPP)|sum(SSP)|sum(NP)|sum(NA)|sum(DCP)|sum(TSP)|sum(DAP)|sum(AP54)|
+----+--------+--------+---------+--------+-------+-------+--------+--------+--------+---------+
|2010|   52802|   23505|   143450|       0|  20799| 155150|   76430|  762722| 1276660|  1212633|
|2011|   45270|   17151|    84660|       0|   8054|  79839|   54680|  408928|  428420|   518219|
|2012|   31870|   14438|   116695|       0|  11022| 171737|   57480|  494311|  666025|   777012|
|2013|   11260|   17620|   127449|       0|   9127| 131246|   46170|  518043|  823537|   747793|
|2014|   21920|    7527|   124385|       0|  12366| 158112|   60470|  482314|  587528|   633027|
|2015|    9670|    5836|    87060|       0|   5605| 149257|   67290|  261797|  369518|   475457|
|2016|   12510|   11497|    67980|       0|  11582| 166339|   63160|  385043|  569800|   599085|
|2017|    9383|   11614|    83

In [14]:
q3b_v2 = dfP.groupBy(dfP['year'][7:10]).agg({'NPK':'sum', 'G27':'sum', 'STPP':'sum', 'SSP':'sum', 'NP':'sum', 'NA':'sum', 'DCP':'sum', 'TSP':'sum', 'DAP':'sum', 'AP54':'sum'}).withColumnRenamed('substring(year, 7, 10)','year').orderBy('year')
q3b_v2.show()

+----+---------+--------+-------+--------+--------+--------+-------+--------+---------+--------+
|year|sum(STPP)|sum(NPK)|sum(NA)|sum(DCP)|sum(TSP)|sum(DAP)|sum(NP)|sum(SSP)|sum(AP54)|sum(G27)|
+----+---------+--------+-------+--------+--------+--------+-------+--------+---------+--------+
|2010|   143450|   52802| 155150|   76430|  762722| 1276660|  20799|       0|  1212633|   23505|
|2011|    84660|   45270|  79839|   54680|  408928|  428420|   8054|       0|   518219|   17151|
|2012|   116695|   31870| 171737|   57480|  494311|  666025|  11022|       0|   777012|   14438|
|2013|   127449|   11260| 131246|   46170|  518043|  823537|   9127|       0|   747793|   17620|
|2014|   124385|   21920| 158112|   60470|  482314|  587528|  12366|       0|   633027|    7527|
|2015|    87060|    9670| 149257|   67290|  261797|  369518|   5605|       0|   475457|    5836|
|2016|    67980|   12510| 166339|   63160|  385043|  569800|  11582|       0|   599085|   11497|
|2017|    83730|    9383| 1705

In [15]:
q3b_v3 = dfP.groupBy(dfP['year'][7:10]).sum().withColumnRenamed('substring(year, 7, 10)','year').orderBy('year')
q3b_v3.show()

+----+--------+--------+---------+--------+-------+-------+--------+--------+--------+---------+
|year|sum(NPK)|sum(G27)|sum(STPP)|sum(SSP)|sum(NP)|sum(NA)|sum(DCP)|sum(TSP)|sum(DAP)|sum(AP54)|
+----+--------+--------+---------+--------+-------+-------+--------+--------+--------+---------+
|2010|   52802|   23505|   143450|       0|  20799| 155150|   76430|  762722| 1276660|  1212633|
|2011|   45270|   17151|    84660|       0|   8054|  79839|   54680|  408928|  428420|   518219|
|2012|   31870|   14438|   116695|       0|  11022| 171737|   57480|  494311|  666025|   777012|
|2013|   11260|   17620|   127449|       0|   9127| 131246|   46170|  518043|  823537|   747793|
|2014|   21920|    7527|   124385|       0|  12366| 158112|   60470|  482314|  587528|   633027|
|2015|    9670|    5836|    87060|       0|   5605| 149257|   67290|  261797|  369518|   475457|
|2016|   12510|   11497|    67980|       0|  11582| 166339|   63160|  385043|  569800|   599085|
|2017|    9383|   11614|    83

In [16]:
q3c = spark.sql('''select substr(year,7) as year, 
                (sum(NPK)+ sum(G27)+ sum(STPP)+ sum(SSP)+ sum(NP)+ sum(NA)+ sum(DCP)+ sum(TSP)+ sum(DAP)+ sum(AP54)) as Total 
                from productions group by (substr(year,7)) order by 1''')
q3c.show()

+----+-------+
|year|  Total|
+----+-------+
|2010|3724151|
|2011|1645221|
|2012|2340590|
|2013|2432245|
|2014|2087649|
|2015|1431490|
|2016|1886996|
|2017|1779081|
|2018|1375944|
|2019|1437747|
|2020| 887328|
+----+-------+



In [18]:
q3d = spark.sql('''select substr(year,7) as year, sum(NPK), sum(G27), sum(STPP), sum(NA), sum(DCP), sum(TSP), sum(DAP), sum(AP54) 
                  from ventes group by (substr(year,7)) order by 1''')
q3d.show()

+----+--------+--------+---------+-------+--------+--------+--------+---------+
|year|sum(NPK)|sum(G27)|sum(STPP)|sum(NA)|sum(DCP)|sum(TSP)|sum(DAP)|sum(AP54)|
+----+--------+--------+---------+-------+--------+--------+--------+---------+
|2010|   52000|   27322|   137202|      0|   54675|  712469| 1157379|   527250|
|2011|   44253|   16954|    82675|      0|   28338|  361326|  354404|   249606|
|2012|   33931|   14199|   108286|   3600|   34100|  446345|  558778|   360095|
|2013|    7960|   10250|   126341|   6577|   20980|  484456|  634325|   284611|
|2014|   21155|    7210|   121362|   1700|   42047|  524731|  503963|   247427|
|2015|   11412|    7800|    81264|      0|   39500|  282673|  223786|   136077|
|2016|   14242|   18915|    72627|      0|   36400|  385428|  560364|   297803|
|2017|    9148|    8490|    75575|      0|   12000|  252480|  440065|   229448|
|2018|   17075|    6450|    48911|   5000|   40356|  291571|  248846|   143407|
|2019|   14300|    5753|    50573|   400

In [17]:
q3d_V2 = dfV.groupBy(dfV['year'][7:10]).sum().withColumnRenamed('substring(year, 7, 10)','year').orderBy('year')
q3d_V2.show()

+----+--------+--------+---------+-------+--------+--------+--------+---------+
|year|sum(NPK)|sum(G27)|sum(STPP)|sum(NA)|sum(DCP)|sum(TSP)|sum(DAP)|sum(AP54)|
+----+--------+--------+---------+-------+--------+--------+--------+---------+
|2010|   52000|   27322|   137202|      0|   54675|  712469| 1157379|   527250|
|2011|   44253|   16954|    82675|      0|   28338|  361326|  354404|   249606|
|2012|   33931|   14199|   108286|   3600|   34100|  446345|  558778|   360095|
|2013|    7960|   10250|   126341|   6577|   20980|  484456|  634325|   284611|
|2014|   21155|    7210|   121362|   1700|   42047|  524731|  503963|   247427|
|2015|   11412|    7800|    81264|      0|   39500|  282673|  223786|   136077|
|2016|   14242|   18915|    72627|      0|   36400|  385428|  560364|   297803|
|2017|    9148|    8490|    75575|      0|   12000|  252480|  440065|   229448|
|2018|   17075|    6450|    48911|   5000|   40356|  291571|  248846|   143407|
|2019|   14300|    5753|    50573|   400

In [23]:
q3b=q3b.toDF(*['year', 'NPK', 'G27', 'STPP', 'SSP', 'NP', 'NA', 'DCP', 'TSP', 'DAP', 'AP54'])
q3d = q3d.toDF(*['year','NPK','G27','STPP','NA','DCP','TSP','DAP','AP54'])

In [24]:
q3b.join(q3d, on='year' ).select(
    'year',q3b.NPK-q3d.NPK, q3b.G27-q3d.G27,q3b.STPP-q3d.STPP,q3b.NA-q3d.NA,
    q3b.DCP-q3d.DCP,q3b.TSP-q3d.TSP,q3b.DAP-q3d.DAP, q3b.AP54-q3d.AP54).show()

+----+-----------+-----------+-------------+---------+-----------+-----------+-----------+-------------+
|year|(NPK - NPK)|(G27 - G27)|(STPP - STPP)|(NA - NA)|(DCP - DCP)|(TSP - TSP)|(DAP - DAP)|(AP54 - AP54)|
+----+-----------+-----------+-------------+---------+-----------+-----------+-----------+-------------+
|2016|      -1732|      -7418|        -4647|   166339|      26760|       -385|       9436|       301282|
|2012|      -2061|        239|         8409|   168137|      23380|      47966|     107247|       416917|
|2020|      -1660|      -1207|        11217|    61737|      12620|      -1268|      14890|       183694|
|2019|      -1215|       6674|        -2913|   156425|       6177|      -7095|     106232|       293704|
|2017|        235|       3124|         8155|   170519|      17070|      47257|     124608|       367975|
|2014|        765|        317|         3023|   156412|      18423|     -42417|      83565|       385600|
|2013|       3300|       7370|         1108|   124669| 